
#  オペレーティングシステム 演習 02
#  プロセス


名前と学生証番号を書け. Enter your name and student ID.

 * 名前 Name:金井優希
 * 学生証番号 Student ID:249217


# 1. プロセス関連コマンド

## 1-1. ps auxww
* ps は現存するプロセスを表示するコマンド
* 話せば長い(詳細はmanページを参照)が, auxwwですべてのプロセスがコマンドラインとともに表示される

* 以下によりシステムのすべてのプロセスが表示される
* 長過ぎる出力がうっとおしければファイルに出力するコマンドに書き換えて実行しなおせば良い
* 例えば
```
ps auxww > ps.txt
```

In [2]:
ps auxww

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root           1  0.0  0.0 167912 13620 ?        Ss   Sep30  14:33 /sbin/init
root           2  0.0  0.0      0     0 ?        S    Sep30   0:04 [kthreadd]
root           3  0.0  0.0      0     0 ?        I<   Sep30   0:00 [rcu_gp]
root           4  0.0  0.0      0     0 ?        I<   Sep30   0:00 [rcu_par_gp]
root           5  0.0  0.0      0     0 ?        I<   Sep30   0:00 [slub_flushwq]
root           6  0.0  0.0      0     0 ?        I<   Sep30   0:00 [netns]
root           8  0.0  0.0      0     0 ?        I<   Sep30   0:00 [kworker/0:0H-events_highpri]
root          11  0.0  0.0      0     0 ?        I<   Sep30   0:00 [mm_percpu_wq]
root          12  0.0  0.0      0     0 ?        S    Sep30   0:00 [rcu_tasks_rude_]
root          13  0.0  0.0      0     0 ?        S    Sep30   0:00 [rcu_tasks_trace]
root          14  0.0  0.0      0     0 ?        S    Sep30   0:18 [ksoftirqd/0]
root          15  0.0  0.0


# 2. fork
* forkはUnixでプロセスを生成する手段
* プロセスを生成 = (例えば実行するファイルを指定して)プログラムを起動するということかと思いきやそうではなく, forkは何の引数もとらず, 呼び出したプロセスのコピーを作るというもの
* 以下は全く役に立たないが, ともかくforkが何をするシステムコールかを教えてくれるプログラム

* OSのシステムコールは特定のプログラミング言語には依存していない
* そこで意味がある場合, なるべく同じことをPythonとCの両方で示すことにする

## 2-1. C

In [3]:
%%writefile fork.c
#include <stdio.h>
#include <unistd.h>

int main() {
  printf("%d : before fork\n", getpid());
  fflush(stdout);
  fork();           /* 現プロセスをコピー */
  printf("%d : after fork\n", getpid());
  return 0;
}

Overwriting fork.c


In [4]:
gcc -Wall fork.c -o fork

In [5]:
./fork

1241030 : before fork
1241030 : after fork
1241031 : after fork


## 2-2. Python

In [6]:
%%writefile fork.py
import os

print(f"{os.getpid()} : before fork", flush=True)
os.fork()
print(f"{os.getpid()} : after fork")

Overwriting fork.py


In [7]:
# 何も表示されない場合, もう一度実行せよ (Jupyter環境の不具合)
python3 fork.py

1241032 : before fork
1241032 : after fork
1241033 : after fork


# 3. 親と子で違う動作をする例
* forkを使うもう少し完結した例
* 親と子で違う動作をする

## 3-1. C

In [8]:
%%writefile fork_pc.c
#include <err.h>
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>

int main() {
  pid_t pid = fork();           /* 現プロセスをコピー */
  if (pid == -1) {
    err(1, "fork");
  } else if (pid == 0) {        /* 新しいプロセス(子プロセス) */
    for (int i = 0; i < 5; i++) {
      printf("child  %d: %d\n", getpid(), i);
      fflush(stdout);
      usleep(100 * 1000);
    }
  } else {                      /* 元いたプロセス(親プロセス)
                                   forkの返り値は子プロセスのプロセスID */
    for (int i = 0; i < 5; i++) {
      printf("parent %d: %d\n", getpid(), i);
      fflush(stdout);
      usleep(100 * 1000);
    }
  }
  return 0;
}

Overwriting fork_pc.c


In [9]:
gcc -Wall fork_pc.c -o fork_pc

In [10]:
./fork_pc

parent 1241043: 0
child  1241044: 0
parent 1241043: 1
child  1241044: 1
parent 1241043: 2
child  1241044: 2
parent 1241043: 3
child  1241044: 3
parent 1241043: 4
child  1241044: 4


## 3-2. Python

In [11]:
%%writefile fork_pc.py
import os
import time

pid = os.fork()
if pid == 0:
    # 新しいプロセス(子プロセス)
    for i in range(5):
        print(f"child  {os.getpid()}: {i}", flush=True)
        time.sleep(0.1)
else:
    # 元いたプロセス(親プロセス) forkの返り値は子プロセスのプロセスID
    for i in range(5):
        print(f"parent {os.getpid()}: {i}", flush=True)
        time.sleep(0.1)

Writing fork_pc.py


In [12]:
python3 fork_pc.py

parent 1241045: 0
child  1241046: 0
parent 1241045: 1
child  1241046: 1
parent 1241045: 2
child  1241046: 2
parent 1241045: 3
child  1241046: 3
parent 1241045: 4
child  1241046: 4


# 4. wait
* forkを使うさらにまともな例
* 親が子のwait処理をする

## 4-1. C

In [13]:
%%writefile fork_wait.c
#include <err.h>
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <sys/types.h>
#include <sys/wait.h>

int main() {
  pid_t pid = fork();
  if (pid == -1) {
    err(1, "fork");
  } else if (pid == 0) {          /* child */
    for (int i = 0; i < 5; i++) {
      printf("%d: %d\n", getpid(), i);
      fflush(stdout);
      usleep(100 * 1000);
    }
    return 123;
  } else {
    int ws;
    printf("parent: wait for child (pid = %d) to finish\n", pid);
    fflush(stdout);
    pid_t cid = waitpid(pid, &ws, 0);
    if (cid == -1) err(1, "waitpid");
    if (WIFEXITED(ws)) {
      printf("exited, status=%d\n", WEXITSTATUS(ws));
      fflush(stdout);
    } else if (WIFSIGNALED(ws)) {
      printf("killed by signal %d\n", WTERMSIG(ws));
      fflush(stdout);
    }
  }
  return 0;
}

Writing fork_wait.c


In [14]:
gcc -Wall fork_wait.c -o fork_wait

In [15]:
./fork_wait

1241057: 0
parent: wait for child (pid = 1241057) to finish
1241057: 1
1241057: 2
1241057: 3
1241057: 4
exited, status=123


## 4-2. Python

In [16]:
%%writefile fork_wait.py
import os
import time

pid = os.fork()
if pid == 0:
    # 新しいプロセス(子プロセス)
    for i in range(5):
        print(f"child  {os.getpid()}: {i}", flush=True)
        time.sleep(0.1)
else:
    # 元いたプロセス(親プロセス) forkの返り値は子プロセスのプロセスID
    print(f"parent: wait for child (pid = {pid}) to finish", flush=True)
    cid, ws = os.waitpid(pid, 0)
    if os.WIFEXITED(ws):
        print(f"exited, status={os.WEXITSTATUS(ws)}", flush=True)
    elif os.WIFSIGNALED(ws):
        print(f"killed by signal {os.WTERMSIG(ws)}", flush=True)


Writing fork_wait.py


In [17]:
python3 fork_wait.py

parent: wait for child (pid = 1241060) to finish
child  1241060: 0
child  1241060: 1
child  1241060: 2
child  1241060: 3
child  1241060: 4
exited, status=0


# 5. exec

* execは指定したプログラムを実行するシステムコール
* 呼び出したプロセスをそのまま, 指定したプログラムを先頭から実行するものに「変える」というイメージ
* execがプロセスを生成するのではないので注意
* execはあくまで呼び出したプロセスを, これまでのことをすべて忘れて所定のことをするプロセスに「変身させる」
* なお, execという名のシステムコールが実在するのではなく, execv, execveなど色々な変種の総称
* 以下は ls -l を実行するプロセスを作るプログラム

## 5-1. C

In [18]:
%%writefile fork_execv.c
#include <err.h>
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <sys/types.h>
#include <sys/wait.h>

extern char ** environ;

int main() {
  pid_t pid = fork();           /* 現プロセスをコピー */
  if (pid == -1) {
    err(1, "fork");
  } else if (pid == 0) {        /* 新しいプロセス(子プロセス) */
    char * const argv[] = { "/bin/ls", "-l", 0 };
    execv(argv[0], argv);
    /* 成功すればexecveはリターンしない.
       i.e., リターンしたらエラー */
    err(1, "execve");
  } else {
    int ws;
    pid_t cid = waitpid(pid, &ws, 0);
    if (cid == -1) err(1, "waitpid");
    if (WIFEXITED(ws)) {
      printf("exited, status=%d\n", WEXITSTATUS(ws));
      fflush(stdout);
    } else if (WIFSIGNALED(ws)) {
      printf("killed by signal %d\n", WTERMSIG(ws));
      fflush(stdout);
    }
  }
  return 0;
}

Writing fork_execv.c


In [19]:
gcc -Wall fork_execv.c -o fork_execv

In [20]:
./fork_execv

total 140
-rwxr-xr-x 1 u24280 u24280 16128 Oct 19 22:01 fork
-rw-r--r-- 1 u24280 u24280   222 Oct 19 22:00 fork.c
-rwxr-xr-x 1 u24280 u24280 16272 Oct 19 22:01 fork_execv
-rw-r--r-- 1 u24280 u24280   874 Oct 19 22:01 fork_execv.c
-rwxr-xr-x 1 u24280 u24280 16216 Oct 19 22:01 fork_pc
-rw-r--r-- 1 u24280 u24280   722 Oct 19 22:01 fork_pc.c
-rw-r--r-- 1 u24280 u24280   406 Oct 19 22:01 fork_pc.py
-rw-r--r-- 1 u24280 u24280   108 Oct 19 22:01 fork.py
-rwxr-xr-x 1 u24280 u24280 16312 Oct 19 22:01 fork_wait
-rw-r--r-- 1 u24280 u24280   804 Oct 19 22:01 fork_wait.c
-rw-r--r-- 1 u24280 u24280   594 Oct 19 22:01 fork_wait.py
-rw-r--r-- 1 u24280 u24280 48643 Oct 19 21:55 os02_process.sos.ipynb
exited, status=0


## 5-2. Python

In [21]:
%%writefile fork_execv.py
import os

pid = os.fork()
if pid == 0:                    # 新しいプロセス(子プロセス)
    argv = [ "/bin/ls", "-l" ]
    os.execv(argv[0], argv)
else:
    cid, ws = os.waitpid(pid, 0)
    if os.WIFEXITED(ws):
        print(f"exited, status={os.WEXITSTATUS(ws)}", flush=True)
    elif os.WIFSIGNALED(ws):
        print(f"killed by signal {os.WTERMSIG(ws)}", flush=True)

Writing fork_execv.py


In [22]:
python3 fork_execv.py

total 144
-rwxr-xr-x 1 u24280 u24280 16128 Oct 19 22:01 fork
-rw-r--r-- 1 u24280 u24280   222 Oct 19 22:00 fork.c
-rwxr-xr-x 1 u24280 u24280 16272 Oct 19 22:01 fork_execv
-rw-r--r-- 1 u24280 u24280   874 Oct 19 22:01 fork_execv.c
-rw-r--r-- 1 u24280 u24280   383 Oct 19 22:01 fork_execv.py
-rwxr-xr-x 1 u24280 u24280 16216 Oct 19 22:01 fork_pc
-rw-r--r-- 1 u24280 u24280   722 Oct 19 22:01 fork_pc.c
-rw-r--r-- 1 u24280 u24280   406 Oct 19 22:01 fork_pc.py
-rw-r--r-- 1 u24280 u24280   108 Oct 19 22:01 fork.py
-rwxr-xr-x 1 u24280 u24280 16312 Oct 19 22:01 fork_wait
-rw-r--r-- 1 u24280 u24280   804 Oct 19 22:01 fork_wait.c
-rw-r--r-- 1 u24280 u24280   594 Oct 19 22:01 fork_wait.py
-rw-r--r-- 1 u24280 u24280 48643 Oct 19 21:55 os02_process.sos.ipynb
exited, status=0


# 6. execvp
* execv関数では実行したいコマンド(ls)のファイル名(/bin/ls)を指定しなくてはならない
* 普段シェルでコマンドを実行する際は ls と打つだけで実行できているのは, シェルがPATHという環境変数を見てコマンドを探してくれているから
* 以下でPATHという環境変数の中身が表示できる

In [23]:
echo $PATH

/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/snap/bin



* シェルは, PATHに指定されているディレクトリを順に見ていって, lsという名前のファイルが見つかったらそれを実行する
* 同じことはexecの変種 execvp という関数を呼べばやってくれる

## 6-1. C

In [24]:
%%writefile fork_execvp.c
#include <err.h>
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <sys/types.h>
#include <sys/wait.h>

extern char ** environ;

int main() {
  pid_t pid = fork();           /* 現プロセスをコピー */
  if (pid == -1) {
    err(1, "fork");
  } else if (pid == 0) {        /* 新しいプロセス(子プロセス) */
    char * const argv[] = { "ls", "-l", 0 };
    execvp(argv[0], argv);
    /* 成功すればexecveはリターンしない.
       i.e., リターンしたらエラー */
    err(1, "execve");
  } else {
    int ws;
    pid_t cid = waitpid(pid, &ws, 0);
    if (cid == -1) err(1, "waitpid");
    if (WIFEXITED(ws)) {
      printf("exited, status=%d\n", WEXITSTATUS(ws));
      fflush(stdout);
    } else if (WIFSIGNALED(ws)) {
      printf("killed by signal %d\n", WTERMSIG(ws));
      fflush(stdout);
    }
  }
  return 0;
}

Writing fork_execvp.c


In [25]:
gcc -Wall fork_execvp.c -o fork_execvp

In [26]:
./fork_execvp

total 164
-rwxr-xr-x 1 u24280 u24280 16128 Oct 19 22:01 fork
-rw-r--r-- 1 u24280 u24280   222 Oct 19 22:00 fork.c
-rwxr-xr-x 1 u24280 u24280 16272 Oct 19 22:01 fork_execv
-rw-r--r-- 1 u24280 u24280   874 Oct 19 22:01 fork_execv.c
-rwxr-xr-x 1 u24280 u24280 16272 Oct 19 22:01 fork_execvp
-rw-r--r-- 1 u24280 u24280   870 Oct 19 22:01 fork_execvp.c
-rw-r--r-- 1 u24280 u24280   383 Oct 19 22:01 fork_execv.py
-rwxr-xr-x 1 u24280 u24280 16216 Oct 19 22:01 fork_pc
-rw-r--r-- 1 u24280 u24280   722 Oct 19 22:01 fork_pc.c
-rw-r--r-- 1 u24280 u24280   406 Oct 19 22:01 fork_pc.py
-rw-r--r-- 1 u24280 u24280   108 Oct 19 22:01 fork.py
-rwxr-xr-x 1 u24280 u24280 16312 Oct 19 22:01 fork_wait
-rw-r--r-- 1 u24280 u24280   804 Oct 19 22:01 fork_wait.c
-rw-r--r-- 1 u24280 u24280   594 Oct 19 22:01 fork_wait.py
-rw-r--r-- 1 u24280 u24280 48643 Oct 19 21:55 os02_process.sos.ipynb
exited, status=0


## 6-2. Python

In [27]:
%%writefile fork_execvp.py
import os

pid = os.fork()
if pid == 0:                    # 新しいプロセス(子プロセス)
    argv = [ "ls", "-l" ]
    os.execvp(argv[0], argv)
else:
    cid, ws = os.waitpid(pid, 0)
    if os.WIFEXITED(ws):
        print(f"exited, status={os.WEXITSTATUS(ws)}", flush=True)
    elif os.WIFSIGNALED(ws):
        print(f"killed by signal {os.WTERMSIG(ws)}", flush=True)

Writing fork_execvp.py


In [28]:
python3 fork_execvp.py

total 168
-rwxr-xr-x 1 u24280 u24280 16128 Oct 19 22:01 fork
-rw-r--r-- 1 u24280 u24280   222 Oct 19 22:00 fork.c
-rwxr-xr-x 1 u24280 u24280 16272 Oct 19 22:01 fork_execv
-rw-r--r-- 1 u24280 u24280   874 Oct 19 22:01 fork_execv.c
-rwxr-xr-x 1 u24280 u24280 16272 Oct 19 22:01 fork_execvp
-rw-r--r-- 1 u24280 u24280   870 Oct 19 22:01 fork_execvp.c
-rw-r--r-- 1 u24280 u24280   379 Oct 19 22:01 fork_execvp.py
-rw-r--r-- 1 u24280 u24280   383 Oct 19 22:01 fork_execv.py
-rwxr-xr-x 1 u24280 u24280 16216 Oct 19 22:01 fork_pc
-rw-r--r-- 1 u24280 u24280   722 Oct 19 22:01 fork_pc.c
-rw-r--r-- 1 u24280 u24280   406 Oct 19 22:01 fork_pc.py
-rw-r--r-- 1 u24280 u24280   108 Oct 19 22:01 fork.py
-rwxr-xr-x 1 u24280 u24280 16312 Oct 19 22:01 fork_wait
-rw-r--r-- 1 u24280 u24280   804 Oct 19 22:01 fork_wait.c
-rw-r--r-- 1 u24280 u24280   594 Oct 19 22:01 fork_wait.py
-rw-r--r-- 1 u24280 u24280 48643 Oct 19 21:55 os02_process.sos.ipynb
exited, status=0


* なお, PATHの中身を見て, コマンド名に対応するファイル名を表示してくれるのが which というコマンド
* コマンドを打ち込んで実行されているファイルがどこにあるのかを知りたいときに使う
* Linux, Macを使っている人は普段使っているプログラム, firefox, zoomなどがどこにあるのかを探ってみよ

In [30]:
which firefox
which zoom

: 1

# 7. forkにまつわる悲劇
* 以下のようなプログラムを書いたらプロセスはいくつ生成されるのだろう?
* そして以下を実行するとどんな表示が出てくるか?
* 頭で予想してから実行してみよ
* n=100としたら何が起こるか(<- 決してやってはいけない)?

## 7-1. C

In [40]:
%%writefile fork_n.c
#include <stdio.h>
#include <unistd.h>

int main() {
  int n = 1;
  for (int i = 0; i < n; i++) {
    pid_t cid = fork();
    printf("%d -> %d: %d\n", getpid(), cid, i);
    fflush(stdout);
  }
  return 0;
}

Overwriting fork_n.c


In [41]:
gcc -Wall fork_n.c -o fork_n

* 以下を実行して (左側のファイル一覧から) out.txt を開いて見てみよ
* プログラムと結果を見て, このプログラムを実行した結果何が起きたのかを考えよ (わかりにくければ n を小さくして)

In [42]:
./fork_n > out.txt

## 7-2. Python

In [46]:
%%writefile fork_n.py
import os

n = 5
for i in range(5):
    cid = os.fork()
    print(f"{os.getpid()} -> {cid}", flush=True)

Writing fork_n.py


* 以下を実行して (左側のファイル一覧から) out.txt を開いて見てみよ
* プログラムと結果を見て, このプログラムを実行した結果何が起きたのかを考えよ (わかりにくければ n を小さくして)

In [47]:
python3 fork_n.py > out.txt

* 注: > out.txt なしで直接表示することもできるが, 結果をすべて出力てくれないことがしばしばある. これは多分, Jupyterのbashカーネルのバグ
* 多分以下のように, このセルの終了まで間を作ると全部出力してくれる(Jupyterのバグを回避しているだけで全く本質的なことではない. 端末で実行すればこんなことをする必要はない)

In [48]:
./fork_n
sleep 1 

1250589 -> 1250590: 0
1250589 -> 1250591: 1
1250589 -> 1250592: 2
1250589 -> 1250593: 3
1250590 -> 0: 0
1250589 -> 1250594: 4
1250592 -> 0: 2
1250591 -> 0: 1
1250592 -> 1250596: 3
1250590 -> 1250595: 1
1250593 -> 0: 3
1250592 -> 1250597: 4
1250594 -> 0: 4
1250591 -> 1250598: 2
1250597 -> 0: 4
1250596 -> 0: 3
1250590 -> 1250599: 2
1250595 -> 0: 1
1250593 -> 1250600: 4
1250598 -> 0: 2
1250596 -> 1250601: 4
1250599 -> 0: 2
1250591 -> 1250602: 3
1250590 -> 1250603: 3
1250595 -> 1250604: 2
1250600 -> 0: 4
1250601 -> 0: 4
1250598 -> 1250605: 3
1250591 -> 1250606: 4
1250602 -> 0: 3
1250590 -> 1250608: 4
1250599 -> 1250607: 3
1250605 -> 0: 3
1250595 -> 1250609: 3
1250604 -> 0: 2
1250608 -> 0: 4
1250603 -> 0: 3
1250598 -> 1250610: 4
1250606 -> 0: 4
1250607 -> 0: 3
1250599 -> 1250611: 4
1250602 -> 1250612: 4
1250609 -> 0: 3
1250595 -> 1250613: 4
1250605 -> 1250614: 4
1250612 -> 0: 4
1250610 -> 0: 4
1250603 -> 1250615: 4
1250604 -> 1250616: 3
1250611 -> 0: 4
1250614 -> 0: 4
1250607 -> 1250617: 4


# <font color="green"> Problem 1 :  fork, exec, waitの練習</font>
以下を行うプログラムを書け

1. 時刻をナノ秒単位で取得($t_0$とする)
2. 以下を多数回($n$回)繰り返す
 * 子プロセスをforkする
  * 子プロセスはただちに ./do_nothing という, 何もしないプログラムをexecする
  * 親プロセスはただちに子プロセスの終了をwaitする
3. 時刻をナノ秒単位で取得($t_1$とする)
4. 1回あたりの時間($(t_1 - t_0)/n$) をナノ秒単位で出力

* do_nothingは以下のような, 何もしないプログラム

In [104]:
%%writefile do_nothing.c
int main() {
    return 0;
}

Overwriting do_nothing.c


In [105]:
gcc -Wall do_nothing.c -o do_nothing

* $n$はコマンドラインから取得できるようにする
* 以下のコードを修正して上記を達成せよ(PythonとC両方)

## 7-3. C

In [120]:

%%writefile time_fork_exec_wait.c
#include <err.h>
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <time.h>
#include <sys/types.h>
#include <sys/wait.h>

long cur_time() {
  struct timespec ts[1];
  clock_gettime(CLOCK_REALTIME, ts);
  return ts->tv_sec * 1000000000L + ts->tv_nsec;
}

int main(int argc, char ** argv) {
    int n = (argc > 1 ? atoi(argv[1]) : 5);
    long t0 = cur_time();

    for (int i=0;i<n;i++){
        pid_t pid = fork();
        if (pid == -1) {
            err(1, "fork");
        } else if (pid == 0) {
            char* const argv[] = {"./do_nothing"};
            execv(argv[0], argv);
            exit(0);
        } else {
            int ws;
            pid_t cid = waitpid(pid, &ws, 0);
            if (cid == -1) err(1, "waitpid");
        }
    }

    long t1 = cur_time();
    long dt = t1 - t0;
    printf("%ld nsec to fork-exec-wait %d processes (%ld nsec/proc)\n", dt, n, dt / n);
    return 0;
}

Overwriting time_fork_exec_wait.c


In [121]:
gcc -Wall time_fork_exec_wait.c -o time_fork_exec_wait

* 同様に以下のコマンドラインを色々変更して, 1回あたりの時間を計測せよ

In [122]:
./time_fork_exec_wait 10

6722782 nsec to fork-exec-wait 10 processes (672278 nsec/proc)


## 7-4. Python

In [126]:

%%writefile time_fork_exec_wait.py
import os
import sys
import time

def cur_time():
    return int(time.time() * 1.0e9)

def main():
    n = int(sys.argv[1]) if 1 < len(sys.argv) else 5
    t0 = cur_time()

    for i in range(n):
        pid = os.fork()
        if pid == 0:
            argv = ["./do_nothing"]
            os.execv(argv[0], argv)
            sys.exit()
        else:
            os.waitpid(pid, 0)
  
    t1 = cur_time()
    dt = t1 - t0
    print(f"{dt} nsec to fork-exec-wait {n} processes ({dt / n} nsec/proc)")

main()

Overwriting time_fork_exec_wait.py


* 以下のコマンドラインを色々変更して, 1回あたりの時間を計測せよ
* これは概ね, fork + exec + exit + wait の時間 (プログラムを起動して終了するまでにかかる最小の時間)を計測していることに相当する
* 正しく動いているかを確認するために, do_nothingで何かをprintするとか, time_fork_exec_wait中でwaitpidの結果を表示するようにせよ
* 時間を計測するときはそれらの表示を消すこと(消さないと, 測っているのは出力時間が大半を占めることになる)

In [128]:
python3 time_fork_exec_wait.py 10

24167936 nsec to fork-exec-wait 10 processes (2416793.6 nsec/proc)


# <font color="green"> Problem 2 :  fork + waitの測定</font>
* 子プロセスが do_nothing を exec する代わりに, 直ちにexit した場合の時間(fork + wait の時間)も計測せよ

## 7-5. C

In [110]:

%%writefile time_fork_exit_wait.c
#include <err.h>
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <time.h>
#include <sys/types.h>
#include <sys/wait.h>

long cur_time() {
  struct timespec ts[1];
  clock_gettime(CLOCK_REALTIME, ts);
  return ts->tv_sec * 1000000000L + ts->tv_nsec;
}

int main(int argc, char ** argv) {
    int n = (argc > 1 ? atoi(argv[1]) : 5);
    long t0 = cur_time();
  
    for (int i=0;i<n;i++){
        pid_t pid = fork();
        if (pid == -1) {
            err(1, "fork");
        } else if (pid == 0) {
            exit(0);
        } else {
            int ws;
            pid_t cid = waitpid(pid, &ws, 0);
            if (cid == -1) err(1, "waitpid");
        }
    }

  
    long t1 = cur_time();
    long dt = t1 - t0;
    printf("%ld nsec to fork-wait %d processes (%ld nsec/proc)\n",dt, n, dt / n);
    return 0;
}

Overwriting time_fork_exit_wait.c


In [111]:

gcc -Wall time_fork_exit_wait.c -o time_fork_exit_wait

In [112]:

./time_fork_exit_wait 10

5590582 nsec to fork-wait 10 processes (559058 nsec/proc)


## 7-6. Python

In [129]:

%%writefile time_fork_exit_wait.py
import os
import sys
import time

def cur_time():
    return int(time.time() * 1.0e9)

def main():
    n = int(sys.argv[1]) if 1 < len(sys.argv) else 5
    t0 = cur_time()

  
    for i in range(n):
        pid = os.fork()
        if pid == 0:
            sys.exit()
        else:
            os.waitpid(pid, 0)

  
    t1 = cur_time()
    dt = t1 - t0
    print(f"{dt} nsec to fork-wait {n} processes ({dt / n} nsec/proc)")

main()

Overwriting time_fork_exit_wait.py


In [132]:
python3 time_fork_exit_wait.py 10

100254720 nsec to fork-wait 10 processes (10025472.0 nsec/proc)
